In [1]:
import pandas as pd
import geopandas
import folium
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
from shapely.geometry import Point, Polygon, CAP_STYLE, box

In [114]:
w=[-27.538360488181283, 152.79806779140625,
-27.503650582690096, 152.96423600429688,
-27.65276453925091, 152.97453568691407,
-27.690467333539093, 152.80424760097657
]
a = np.array(w).reshape(4,2)
print('Polygon('+str([(x[1],x[0]) for x in a])+')')

Polygon([(152.79806779140625, -27.538360488181283), (152.96423600429688, -27.503650582690096), (152.97453568691407, -27.65276453925091), (152.80424760097657, -27.690467333539093)])


In [55]:
zones={
'Mel North':Polygon([(144.8628073558594, -37.7212887643389), (145.0111227855469, -37.71585722493781), (145.00974949453126, -37.78752147312994), (144.8628073558594, -37.79294775423262)]),
'Mel West': Polygon([(144.73509129140626, -37.75821267002394), (144.8573141917969, -37.75821267002394), (144.86006077382814, -37.848273056101036), (144.73509129140626, -37.86020052689999)]),
'Mel East':Polygon([(145.05781468007814, -37.78969203338907), (145.20201023671876, -37.79294775423262), (145.193770490625, -37.888385058604605), (145.05918797109376, -37.89055265284611)]),
'Mel Center':Polygon([(144.87791355703126, -37.804884169931675), (145.03034885976564, -37.80162897512093), (145.03584202382814, -37.86020052689999), (144.875166975, -37.85152619406298)]),
'Mel SouthEast':Polygon([(145.00288303945314, -37.900306037210385), (145.14570530507814, -37.90789110935228), (145.18003758046876, -37.99560434902989), (145.0330954417969, -37.99127528948802)]),                     
'Syd North':Polygon([(151.1512687398634, -33.7935992898409), (151.2281730367384, -33.79645241440158), (151.24122619628906, -33.84333046657819), (151.16294171349622, -33.84180431578851)]),
'Syd Center':Polygon([(151.16912152306654, -33.856631164603094), (151.20414044396497, -33.87402091845947), (151.20414044396497, -33.91078430372838), (151.1553886129103, -33.90451569998407)]),
'Syd East':Polygon([(151.25769879357435, -33.874591014351736), (151.20414044396497, -33.95664505114222), (151.2439658834181, -34.00390417629284), (151.28585125939466, -33.85121396050401)]),
'Syd West':Polygon([(151.0523917867384, -33.87060026313806), (151.15401532189466, -33.87060026313806), (151.13272931115247, -33.924744536934675), (151.02767254845716, -33.92417477627145)]),
'Syd Parramatta':Polygon([(150.93291546837904, -33.767346082697074), (151.0523917867384, -33.776478545694935), (151.0523917867384, -33.866609325339375), (150.90476300255872, -33.87060026313806)]),
'Syd South':Polygon([(151.08191754357435, -33.93670863052086), (151.2000205709181, -33.92702354147929), (151.1553886129103, -34.0642209857176), (151.03797223107435, -34.03691302642577)]),
'Bri North':Polygon([(152.96972916835938, -27.32627690929374), (153.08165238613282, -27.331156966171314), (153.12628434414063, -27.421399244879808), (152.9759089779297, -27.426884576089662)]),      
'Bri Center':Polygon([(152.97865555996094, -27.447604478804557), (153.15031693691407, -27.426884576089662), (153.19769547695313, -27.49938720750099), (152.9765956234375, -27.542013525008592)]),
'Bri South':Polygon([(152.9820887875, -27.5505368052517), (153.2011287044922, -27.542013525008592), (153.2121150326172, -27.619916025981073), (152.98964188808594, -27.661279210523194)]),
'Bri West':Polygon([(152.79806779140625, -27.538360488181283), (152.96423600429688, -27.503650582690096), (152.97453568691407, -27.65276453925091), (152.80424760097657, -27.690467333539093)])
}
pivot_z = {'zone':zones.keys(), 'geometry':zones.values()}
zones_gdf = geopandas.GeoDataFrame(pivot_z, crs="EPSG:4326" )

#make it square zone
square = [Polygon([(l,b),(r,b),(r,t),(l,t)]) for l, b, r, t in zones_gdf.geometry.bounds.values.tolist()] 
square_gdf = geopandas.GeoDataFrame(zones_gdf['zone'], geometry=square, crs="EPSG:4326")

tost=square_gdf.to_crs({'init': 'epsg:32633'})
square_gdf['radius']=np.sqrt( tost['geometry'].area/ 10**6/np.pi)
square_gdf['center_longitude']=square_gdf.geometry.centroid.x
square_gdf['center_latitude']=square_gdf.geometry.centroid.y

c:\program files\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
c:\program files\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

c:\program files\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [56]:
square_gdf.to_file('target_zones.geojson', driver="GeoJSON")

In [57]:
square_gdf=geopandas.read_file('target_zones.geojson')

In [58]:
square_gdf

,zone,radius,center_longitude,center_latitude,geometry
0,Mel North,7.501889,144.936965,-37.754402,"POLYGON ((144.86281 -37.79295, 145.01112 -37.7..."
1,Mel West,7.923528,144.797576,-37.809207,"POLYGON ((144.73509 -37.86020, 144.86006 -37.8..."
2,Mel East,8.436563,145.129912,-37.840122,"POLYGON ((145.05781 -37.89055, 145.20201 -37.8..."
3,Mel Center,6.797401,144.955504,-37.830915,"POLYGON ((144.87517 -37.86020, 145.03584 -37.8..."
4,Mel SouthEast,9.078377,145.091460,-37.947955,"POLYGON ((145.00288 -37.99560, 145.18004 -37.9..."
5,Syd North,4.675385,151.196247,-33.818465,"POLYGON ((151.15127 -33.84333, 151.24123 -33.8..."
6,Syd Center,3.589449,151.179765,-33.883708,"POLYGON ((151.15539 -33.91078, 151.20414 -33.9..."
7,Syd East,7.794549,151.244996,-33.927559,"POLYGON ((151.20414 -34.00390, 151.28585 -34.0..."
8,Syd West,5.781608,151.090844,-33.897672,"POLYGON ((151.02767 -33.92474, 151.15402 -33.9..."
9,Syd Parramatta,8.647161,150.978577,-33.818973,"POLYGON ((150.90476 -33.87060, 151.05239 -33.8..."


## Census data: Income, Age (SA2)

In [27]:
df=pd.read_csv('SA2-P02_Selected_Medians_and_Averages-Census_2016.csv')
df.columns = [x.strip().upper() for x in df.columns]
df['SA2_MAIN16'] = df['SA2_MAIN16'].astype(str)

In [ ]:
#http://apps.headwallphotonics.com/

In [28]:
geo_data =geopandas.read_file('./shapefile/sa2_low_resolution.geojson')
geo_data = geo_data[~geo_data.geometry.isna()]
loc = Point(144,-37.8) #long, lat
loc_buffer=loc.buffer(0.5, cap_style=CAP_STYLE.square)
geo_data['in'] = geo_data.geometry.contains(loc)
geo_data['contains'] = geo_data.geometry.apply(lambda x: loc_buffer.contains(x))

geo_data[geo_data['in']]

,SA2_MAIN16,SA2_NAME16,geometry,in,contains
589,201021012,Gordon (Vic.),"POLYGON ((143.94409 -37.57381, 143.94168 -37.5...",True,True


In [59]:
sa2_to_zones =geopandas.tools.sjoin(square_gdf, geo_data, how='left')
zone_income_age = pd.merge(sa2_to_zones, df).groupby(['zone']).mean()[['MEDIAN_AGE_PERSONS','MEDIAN_TOT_FAM_INC_WEEKLY',
       'AVERAGE_HOUSEHOLD_SIZE', 'MEDIAN_TOT_PRSNL_INC_WEEKLY']]

c:\program files\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4283

  """Entry point for launching an IPython kernel.


In [60]:
m = folium.Map(location=[-23, 133], zoom_start=5)

folium.Choropleth(
    geo_data=square_gdf,
    name='choropleth',
    data=zone_income_age.reset_index(),
    columns=['zone', 'MEDIAN_TOT_FAM_INC_WEEKLY'],
    key_on='feature.properties.zone',
    nan_fill_color='white',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Median Total Family Income'
).add_to(m)
folium.GeoJson(data=square_gdf).add_to(m)
folium.LayerControl().add_to(m)
m

## Religion

In [61]:
religon=pd.read_csv('SA3-G14_Religious_Affiliation_by_Sex-Census_2016.csv')
religon.columns = [x.strip().upper() for x in religon.columns]
religon = religon[[ 'SA3_NAME16','TOT_P','CHRISTIANITY_TOT_P','ISLAM_P','HINDUISM_P', 'SB_OSB_NRA_NR_P']]

Index(['SA3_NAME16', 'TOT_P', 'CHRISTIANITY_TOT_P', 'ISLAM_P', 'HINDUISM_P',
       'SB_OSB_NRA_NR_P'],
      dtype='object')

In [63]:
geo_sa3 =geopandas.read_file('./shapefile/sa3_low_resolution.geojson')
geo_sa3 = geo_sa3[~geo_sa3.geometry.isna()]

In [67]:
sa3_to_zones =geopandas.tools.sjoin(square_gdf, geo_sa3, how='left')
zone_religon = pd.merge(sa3_to_zones, religon, on='SA3_NAME16').groupby(['zone']).sum()

c:\program files\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4283

  """Entry point for launching an IPython kernel.


In [69]:
zone_religon.columns

Index(['radius', 'center_longitude', 'center_latitude', 'index_right', 'TOT_P',
       'CHRISTIANITY_TOT_P', 'ISLAM_P', 'HINDUISM_P', 'SB_OSB_NRA_NR_P'],
      dtype='object')

In [72]:
zone_religon['ratio_christian'] = zone_religon.CHRISTIANITY_TOT_P/zone_religon.TOT_P
zone_religon['ratio_islam'] = zone_religon.ISLAM_P/zone_religon.TOT_P
zone_religon['ratio_hindu'] = zone_religon.HINDUISM_P/zone_religon.TOT_P
zone_religon['ratio_no_religon'] = zone_religon.SB_OSB_NRA_NR_P/zone_religon.TOT_P


In [75]:
m = folium.Map(location=[-23, 133], zoom_start=5)

folium.Choropleth(
    geo_data=square_gdf,
    name='choropleth',
    data=zone_religon.reset_index(),
    columns=['zone', 'ratio_christian'],
    key_on='feature.properties.zone',
    nan_fill_color='white',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Ratio of Christian in total population'
).add_to(m)
folium.GeoJson(data=square_gdf).add_to(m)
folium.LayerControl().add_to(m)
m

## Education

In [138]:
edu=pd.read_csv('SA3-based_B16A_Highest_Year_of_School_completed_by_Age_by_Sex_as_at_2011-08-11.csv')
edu.columns = [x.strip().upper() for x in edu.columns]
edu = edu[['M_DNGTS_TOT',
 'M_Y8B_TOT',
 'F_Y8B_TOT',
 'F_DNGTS_TOT',
 'F_Y9E_TOT',
 'P_Y9E_TOT',
 'P_Y10E_TOT',
 'P_Y11E_TOT',
 'P_Y12E_TOT','SA3_NAME11']]
zone_edu = pd.merge(sa3_to_zones, edu, left_on='SA3_NAME16', right_on='SA3_NAME11').groupby(['zone']).sum()

In [144]:
zone_edu['P_TOT']=zone_edu[['M_DNGTS_TOT',
 'M_Y8B_TOT',
 'F_Y8B_TOT',
 'F_DNGTS_TOT',
 'F_Y9E_TOT',
 'P_Y9E_TOT',
 'P_Y10E_TOT',
 'P_Y11E_TOT',
 'P_Y12E_TOT']].sum(axis=1)
zone_edu['avg_study_yrs'] = zone_edu[[
 'M_Y8B_TOT',
 'F_Y8B_TOT',
 'P_Y9E_TOT',
 'P_Y10E_TOT',
 'P_Y11E_TOT',
 'P_Y12E_TOT','P_TOT']].apply(lambda x: (x[0]*4.0 +x[1]*4+x[2]*9+x[3]*10+x[4]*11+x[5]*12)/x[6], axis=1)

In [140]:
zone_edu

,radius,center_longitude,center_latitude,index_right,M_DNGTS_TOT,M_Y8B_TOT,F_Y8B_TOT,F_DNGTS_TOT,F_Y9E_TOT,P_Y9E_TOT,P_Y10E_TOT,P_Y11E_TOT,P_Y12E_TOT,P_TOT,avg_yrs
zone,,,,,,,,,,,,,,,
Bri Center,163.821702,2143.220038,-384.782287,2425,1288,10886,14303,1579,7412,14868,93458,30192,394557,568543,10.968289
Bri North,64.524588,1071.336047,-191.636065,1250,501,6042,8195,606,4156,8285,52751,15674,164692,260902,10.761669
Bri South,133.465750,1684.068121,-303.618110,2035,1857,12164,14823,2398,9957,20273,106811,32821,274821,475925,10.542413
Bri West,119.977062,1375.976716,-248.373531,1744,1335,8981,10438,1634,7645,15820,80977,25726,205247,357803,10.552667
Mel Center,54.379212,1159.644036,-302.647318,995,2083,9120,12837,3290,6534,12672,34782,36887,390319,508524,11.089508
Mel East,75.929067,1306.169212,-340.561101,1152,2500,14455,21045,3664,13168,24895,68599,67395,470389,686110,10.840912
Mel North,67.517002,1304.432686,-339.789622,1099,3600,16751,23326,5777,8594,17034,39881,38101,333610,486674,10.550913
Mel SouthEast,45.391886,725.457302,-189.739776,651,3434,13971,19883,4861,10851,20572,53937,48592,322558,498659,10.558602
Mel West,47.541165,868.785456,-226.855240,859,3714,18013,23098,5479,13561,27398,67125,55381,261131,474900,10.160116


## Overseas vs Local

In [133]:
overseas=pd.read_csv('./ABS_-_Data_by_Region_-_Persons_Born_Overseas__SA3__2011-2016.csv')
overseas.columns = [x.strip().upper() for x in overseas.columns]
overseas=overseas[['POPULATION_OF_PERSONS_BORN_OVERSEAS_CENSUS_PERSONS_NUM','SA3_CODE_2016','SA3_NAME_2016']]
zone_overseas = pd.merge(sa3_to_zones, overseas, left_on='SA3_NAME16', right_on='SA3_NAME_2016').groupby(['zone']).sum()

In [134]:
zone_overseas = pd.merge(zone_overseas, zone_religon['TOT_P'], left_index=True, right_index=True)
zone_overseas['rate_overseas'] = zone_overseas.POPULATION_OF_PERSONS_BORN_OVERSEAS_CENSUS_PERSONS_NUM/zone_overseas.TOT_P

In [149]:
zone_stats = pd.merge(
pd.merge(
    pd.merge(zone_overseas['rate_overseas'], zone_edu['avg_study_yrs'], left_index=True, right_index=True),
    zone_religon[['ratio_christian','ratio_islam','ratio_hindu','ratio_no_religon']], left_index=True, right_index=True),
    zone_income_age[['MEDIAN_AGE_PERSONS','MEDIAN_TOT_FAM_INC_WEEKLY']], left_index=True, right_index=True)

In [152]:
zone_stats.reset_index().to_csv('zone_stats.csv')